<img src="https://raw.githubusercontent.com/afo/data-x-plaksha/master/imgsource/dx_logo.png" align="left"></img><br><br><br><br><br><br><br><br>


## Part 1 - Webscraping covid19 data

**Author List**: Deepankar Singh <br>
Site to scrape covid data: <br>
https://en.wikipedia.org/wiki/COVID-19_pandemic_by_country_and_territory

---
<a id='sec4'></a>
# Problem

Use BeautifulSoup and Requests or Pandas to scrape the table “COVID-19 pandemic by <br>
location” under Statistics / Total cases and deaths on this Wikipedia page: <br>
https://en.wikipedia.org/wiki/COVID-19_pandemic_by_country_and_territory

# Solution

In [452]:
import requests # The requests library is an 
# HTTP library for getting and posting content etc.
import bs4 as bs # BeautifulSoup4 is a Python library 
# for pulling data out of HTML and XML code.
# We can query markup languages for specific content
import pandas as pd
import numpy as np

In [453]:
source = requests.get("https://en.wikipedia.org/wiki/COVID-19_pandemic_by_country_and_territory") 
# a GET request will download the HTML webpage.

In [454]:
# Convert source.content to a beautifulsoup object 
# beautifulsoup can parse (extract specific information) HTML code
soup = bs.BeautifulSoup(source.content, features='html.parser') 
# we pass in the source content
# features specifies what type of code we are parsing, 
# here 'html.parser' specifies that we want beautiful soup to parse HTML code

In [455]:
covid_data = soup.find(id='covid19-container')

In [456]:
covid_table = covid_data.find_all('table')[0]

In [457]:
df = pd.read_html(str(covid_table), header=0)[0]
df = df.drop(df.columns[[0,5]],axis=1)
df.columns=['country', 'cases', 'deaths','recovered']
df.head()

,country,cases,deaths,recovered
0,World[e],79290216,1740423,44652378
1,United States[f],18846059,332283,8280764
2,India,10123778,146756,9693173
3,Brazil,7424430,190032,6448740
4,Russia[g],2963688,53096,2370857


In [458]:
df['country'] = df['country'].str.replace(r"\[.*\]","")
df = df[df['country'] != 'World']
df = df.set_index('country')
df = df[:237]
df

,cases,deaths,recovered
country,,,
United States,18846059,332283,8280764
India,10123778,146756,9693173
Brazil,7424430,190032,6448740
Russia,2963688,53096,2370857
France,2527509,62268,188639
...,...,...,...
Wallis and Futuna,4,0,1
American Samoa,3,0,0
Samoa,2,0,0


#### Drop all the rows that do not contain numerical data

In [459]:
for col in df.columns:
 df = df[pd.to_numeric(df[col], errors='coerce').notnull()]
df

,cases,deaths,recovered
country,,,
United States,18846059,332283,8280764
India,10123778,146756,9693173
Brazil,7424430,190032,6448740
Russia,2963688,53096,2370857
France,2527509,62268,188639
...,...,...,...
Marshall Islands,4,0,2
Wallis and Futuna,4,0,1
American Samoa,3,0,0


#### Convert the datatype of all DataFrame values from objects to integers.

In [460]:
df = df.apply(pd.to_numeric) 
df

,cases,deaths,recovered
country,,,
United States,18846059,332283,8280764
India,10123778,146756,9693173
Brazil,7424430,190032,6448740
Russia,2963688,53096,2370857
France,2527509,62268,188639
...,...,...,...
Marshall Islands,4,0,2
Wallis and Futuna,4,0,1
American Samoa,3,0,0


#### Drop all rows of countries with zero recorded deaths or non-numeric death data.

In [461]:
df = df[ df['deaths'] != 0]
df

,cases,deaths,recovered
country,,,
United States,18846059,332283,8280764
India,10123778,146756,9693173
Brazil,7424430,190032,6448740
Russia,2963688,53096,2370857
France,2527509,62268,188639
...,...,...,...
Northern Mariana Islands,113,2,32
British Virgin Islands,72,1,70
Fiji,46,2,44


#### Create a new column called cases_per_deaths and assign it the value of number of cases divided by deaths.

In [462]:
df = df.assign(cases_per_deaths =  df['cases'] // df['deaths'])
df

,cases,deaths,recovered,cases_per_deaths
country,,,,
United States,18846059,332283,8280764,56
India,10123778,146756,9693173,68
Brazil,7424430,190032,6448740,39
Russia,2963688,53096,2370857,55
France,2527509,62268,188639,40
...,...,...,...,...
Northern Mariana Islands,113,2,32,56
British Virgin Islands,72,1,70,72
Fiji,46,2,44,23


In [463]:
#### Sort the DataFrame so that the countries with the highest number of cases_per_deathscome first. 
#### Print the first 20 rows of your sorted DataFrame

In [464]:
df = df.sort_values(by='cases_per_deaths', ascending=False)
df[:20]

,cases,deaths,recovered,cases_per_deaths
country,,,,
Singapore,58495,29,58332,2017
USS Theodore Roosevelt,1102,1,751,1102
Eritrea,877,1,599,877
Qatar,142605,243,140404,586
Burundi,760,2,687,380
Curaçao,4051,12,2385,337
Botswana,11982,38,11147,315
United Arab Emirates,198435,647,174479,306
Maldives,13537,48,12983,282


###### Countries with a high value of cases per death number indicates that the number of deaths per cases is low for them and countries with a low value of cases per death number means that less number of people died per number of cases.

#### Part 2 - Exploring prediction algorithms on covid19 data

###### Load the CSV on this URL https://covid.ourworldindata.org/data/ecdc/total_deaths.csv into a Pandas DataFrame. 
###### It has data until the end of November (otherwise drop all data after November 29 2020).

In [465]:
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
# Keras specific
import keras
from keras.models import Sequential
from keras.layers import Dense
plt.style.use('classic')

In [466]:
data = pd.read_csv('total_deaths.csv')
data['date'] = pd.to_datetime(data['date'])
data = data.replace(np.nan, 0)
data

,date,World,Afghanistan,Albania,Algeria,Andorra,Angola,Anguilla,Antigua and Barbuda,Argentina,...,Uzbekistan,Vanuatu,Vatican,Venezuela,Vietnam,Wallis and Futuna,Western Sahara,Yemen,Zambia,Zimbabwe
0,2019-12-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,2020-11-25,1410788.0,1725.0,735.0,2309.0,76.0,337.0,0.0,4.0,37432.0,...,605.0,0.0,0.0,876.0,35.0,0.0,1.0,609.0,357.0,274.0
331,2020-11-26,1423371.0,1737.0,743.0,2329.0,76.0,338.0,0.0,4.0,37701.0,...,606.0,0.0,0.0,880.0,35.0,0.0,1.0,611.0,357.0,274.0
332,2020-11-27,1434009.0,1737.0,753.0,2352.0,76.0,340.0,0.0,4.0,37941.0,...,607.0,0.0,0.0,880.0,35.0,0.0,1.0,612.0,357.0,274.0
333,2020-11-28,1444342.0,1752.0,771.0,0.0,76.0,0.0,0.0,4.0,38203.0,...,607.0,0.0,0.0,884.0,35.0,0.0,1.0,614.0,357.0,275.0


#### 1. Plot the total number of deaths over time globally.

In [467]:
plt.figure(figsize=(15, 12))
x = np.array(data['date'])
y = np.array(data['World'])
plt.plot(x, y)
plt.xlabel('Date')
plt.ylabel('Total number of deaths')
plt.title('Global death toll of COVID-19 over time')
plt.show()

#### 2. Use scikit-learn to train a Simple Linear Regression model predicting the number ofglobal coronavirus deaths over the time period in the data set. Fill all NaN values with 0.
#### Plot the predictions of the model in a graph together with the true number of cases.

In [468]:
ord_date = data['date'].map(dt.datetime.toordinal)
X = np.array(ord_date).reshape(-1,1)
Y = data['World']
lr = LinearRegression()
lr.fit(X,Y)
pred = lr.predict(X)

Text(0.5, 1.0, 'Comprision between predictions from a linear regression and the actual gloabla death toll of COVID-19')

In [469]:
plt.figure(figsize=(15, 10))
plt.plot(data['date'],data['World'])
plt.plot(data['date'],pred)
plt.xlabel('Date')
plt.ylabel('Total Number of Deaths')
plt.title('Comprision between predictions from a linear regression and the actual gloabla death toll of COVID-19')

#### 3. Print the mean squared error of your Linear Regression model (as compared to the realvalues in the model training data for the whole time period).

In [470]:
print("MSE for Linear Regression model {}".format(mean_squared_error(pred, data['World'])))

MSE for Linear Regression model 7944480123.264536


#### 4. Train any Regression model of your choice on the number of COVID19 deaths in South Korea.
Use data from the date of the first death up to November 22nd as your training set. Your
test set will contain the data from November 23rd to November 29th (7 days in total).
Use your model to predict the number of deaths in South Korea for the seven days
in the test set (not included in your training set) .
Plot the predictions together with the actual recorded deaths for the dates November
23rd to November 29th, see example plot below.
Print the Mean Squared Error (MSE) of your test set predictions. Full credit is given if
your model has a lower MSE than 53.2.

In [503]:
sk = data[['date','World','South Korea']]

train = sk[sk['date'] < dt.datetime(2020, 11, 23)]
train = train[ sk['South Korea'] > 0.0 ]

test = sk[sk['date'] > dt.datetime(2020, 11, 22)]

train = train.assign(rest =  train['World'] - train['South Korea'])
test = test.assign(rest =  test['World'] - test['South Korea'])

train = train.drop(['date', 'World'], axis='columns')
test = test.drop(['date', 'World'], axis='columns')

train['date_num'] = np.array([i for i in range(train.shape[0])])
test['date_num'] = np.array([i for i in range(train.shape[0], train.shape[0]+7, 1)])

train

<ipython-input-503-c2a5661943c4>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  train = train[ sk['South Korea'] > 0.0 ]


,South Korea,rest,date_num
52,1.0,2246.0,0
53,2.0,2357.0,1
54,5.0,2458.0,2
55,7.0,2612.0,3
56,8.0,2690.0,4
...,...,...,...
323,496.0,1339809.0,271
324,498.0,1350895.0,272
325,501.0,1361602.0,273
326,503.0,1373594.0,274


In [519]:
# X = np.array([i for i in range(train.shape[0])])
# train.drop(['date', 'World'], axis='columns')
X = train.drop(['South Korea','rest'], axis='columns')
Y = train['South Korea']
X

,date_num
52,0
53,1
54,2
55,3
56,4
...,...
323,271
324,272
325,273
326,274


In [526]:
# Define model
model = Sequential()
model.add(Dense(124, input_dim=1, activation= "relu"))
# model.add(Dense(1024, activation= "relu"))
model.add(Dense(512, activation= "relu"))
# model.add(Dense(256, activation= "relu"))
# model.add(Dense(128, activation= "relu"))
model.add(Dense(64, activation= "relu"))
model.add(Dense(32, activation= "relu"))
model.add(Dense(16, activation= "relu"))
model.add(Dense(8, activation= "relu"))
model.add(Dense(4, activation= "relu"))
model.add(Dense(1))
# model.summary() #Print model Summary

In [527]:
from tensorflow.keras.optimizers import Adam, RMSprop
model.compile(loss= "mean_squared_error" , optimizer = Adam(learning_rate = 0.001), metrics=["mean_squared_error"])
model.fit(Y, X, epochs=4450, validation_split=0.2)#,verbose=0)

Epoch 1/4450
7/7 [==============================] - 0s 19ms/step - loss: 16355.4658 - mean_squared_error: 16355.4658 - val_loss: 62774.8203 - val_mean_squared_error: 62774.8203
Epoch 2/4450
7/7 [==============================] - 0s 4ms/step - loss: 16225.5410 - mean_squared_error: 16225.5410 - val_loss: 61723.0703 - val_mean_squared_error: 61723.0703
Epoch 3/4450
7/7 [==============================] - 0s 4ms/step - loss: 16059.4980 - mean_squared_error: 16059.4980 - val_loss: 61586.3945 - val_mean_squared_error: 61586.3945
Epoch 4/4450
7/7 [==============================] - 0s 5ms/step - loss: 16033.9102 - mean_squared_error: 16033.9102 - val_loss: 61515.9453 - val_mean_squared_error: 61515.9453
Epoch 5/4450
7/7 [==============================] - 0s 4ms/step - loss: 16019.9512 - mean_squared_error: 16019.9512 - val_loss: 61532.6055 - val_mean_squared_error: 61532.6055
Epoch 6/4450
7/7 [==============================] - 0s 4ms/step - loss: 16019.0908 - mean_squared_error: 16019.0908 - v

In [525]:
pred_train= model.predict(X)
print(np.sqrt(mean_squared_error(Y,pred_train)))

pred = model.predict(np.array([i for i in range(train.shape[0], train.shape[0]+7, 1)]))
print(mean_squared_error(test['South Korea'],pred))

249.43185908087617
153797.36274691875


In [446]:
test['South Korea'],model.predict([275,276,277,278,279,280,281])

(328    509.0
 329    510.0
 330    513.0
 331    515.0
 332    516.0
 333    522.0
 334    523.0
 Name: South Korea, dtype: float64,
 array([[ 98.82467 ],
        [100.73091 ],
        [102.678635],
        [104.67484 ],
        [106.671005],
        [108.66724 ],
        [110.66343 ]], dtype=float32))